### **Importing Libraries**

In [ ]:
import math
import random
from collections import defaultdict
from typing import List
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import deque
import copy
from Card_and_Deck import *
from GameState import *
from Game_and_Player import *
import time

### **Helper Functions**

In [ ]:

def simulate(game_state):
    """Simulates a random game from this node (play out the game to the end)."""
    game_state_new = copy.deepcopy(game_state)
    starting_player_index = game_state.current_player_index
    starting_player_name = game_state.players[starting_player_index].name
    # rewards = {player.name: 0 for player in game_state_new.players}
    # print("initialized rewards")

    while not game_state_new.is_terminal():
        legal_moves = game_state_new.get_legal_moves()
        if not legal_moves:
            break
        move = random.choice(legal_moves)
        game_state_new.apply_move(move)

        # if len(game_state_new.table_cards) == 3:
        #     winning_card, winner_index = game_state_new.get_winner(game_state_new.table_cards)
        #     winner_name = game_state_new.players[winner_index].name
        #     rewards[winner_name] += 1
        #     game_state_new.tricks_won[winner_name] += 1
        #     print(f"Trick winner: {winner_name}, Rewards: {rewards}")
        #     print(f"trick:{game_state_new.table_cards}")
    rewards=game_state_new.tricks_won

    winner_name = max(rewards, key=rewards.get)  # Getting player name with the most tricks


    #     # Check if Monte Carlo player won
    #     if winner_name == "Alice":
    #         monte_carlo_wins += 1

    # win_rate = monte_carlo_wins / num_games
    # print(f"Monte Carlo Win Rate: {win_rate:.2%}")


    # winner_name = max(rewards, key=rewards.get)  # Get player name with the most tricks
    winner_index = next(i for i, player in enumerate(game_state_new.players) if player.name == winner_name)
    # print(f"Winner: {winner_name} (Index: {winner_index})")

    return winner_index


In [ ]:
def monte_carlo_best_move(game_state, num_simulations=1000):
    legal_moves = game_state.get_legal_moves()
    move_wins = {move: 0 for move in legal_moves}
    # move_trials = {move: 0 for move in legal_moves}

    for move in legal_moves:
        for _ in range(num_simulations):
            sim_game_state = copy.deepcopy(game_state)
            sim_game_state.apply_move(move)

            winner = simulate(sim_game_state)

            if winner == game_state.current_player_index:
                move_wins[move] += 1
            # move_trials[move] += 1

    # Selecting the move with the highest win rate
    best_move = max(legal_moves, key=lambda m: (move_wins[m], m.rank))
    return best_move

In [ ]:
np.random.seed(24)
player_names = ["Alice", "Bob", "Charlie"]
roles = ["Teen", "Do", "Paanch"]
index = [0,1,2]

### **Monte Carlo vs. Random Play (Baseline)**

In [ ]:


def evaluate_monte_carlo(player_names,roles,index,num_games=1000, num_simulations=1000):
    monte_carlo_wins = 0
    win_results = []

    for _ in range(num_games):

        game = Game(player_names, roles,index)
        game_state = GameState(game)

        # game_state = copy.deepcopy(initial_game_state)

        game_state.current_player_index=random.choice(index)



        while not game_state.is_terminal():
            if game_state.current_player_index == 0:  # Monte Carlo player is Player 0
                best_move = monte_carlo_best_move(game_state, num_simulations)
                # print(f"Monte Carlo Best Move: {best_move}")
            else:
                legal_moves = game_state.get_legal_moves()
                best_move = random.choice(legal_moves)

            game_state.apply_move(best_move)

            # print("Table Cards:", game_state.table_cards)
            # print("Current Player Index:", game_state.current_player_index)
            # print("Tricks Won:", game_state.tricks_won)

        rewards=game_state.tricks_won
        print(f"Rewards: {rewards}")


        max_tricks = max(rewards.values())
        potential_winners = [name for name, count in rewards.items() if count == max_tricks]

        winner_name = potential_winners[0]
        print(f"Winner: {winner_name}")

        if winner_name == "Alice":
            monte_carlo_wins += 1
            win_results.append(1)
        else:
            win_results.append(0)

    win_rate = monte_carlo_wins / num_games
    std_dev = np.std(win_results, ddof=1)

    print(f"Monte Carlo vs Random Win Rate: {win_rate:.2%}")
    print(f"Standard Deviation of Wins: {std_dev:.4f}")





In [ ]:
player_names = ["Alice", "Bob", "Charlie"]
roles = ["Teen", "Do", "Paanch"]
index = [0,1,2]

In [ ]:
start=time.time()
evaluate_monte_carlo(player_names,roles,index,num_games=1000, num_simulations=50)
end=time.time()
print(f"Execution Time: {end - start:.4f} seconds")

Rewards: {'Alice': 1, 'Bob': 5, 'Charlie': 4}
Winner: Bob
Rewards: {'Alice': 5, 'Bob': 2, 'Charlie': 3}
Winner: Alice
Rewards: {'Alice': 2, 'Bob': 7, 'Charlie': 1}
Winner: Bob
Rewards: {'Alice': 3, 'Bob': 2, 'Charlie': 5}
Winner: Charlie
Rewards: {'Alice': 4, 'Bob': 4, 'Charlie': 2}
Winner: Alice
Rewards: {'Alice': 4, 'Bob': 2, 'Charlie': 4}
Winner: Alice
Rewards: {'Alice': 4, 'Bob': 4, 'Charlie': 2}
Winner: Alice
Rewards: {'Alice': 3, 'Bob': 3, 'Charlie': 4}
Winner: Charlie
Rewards: {'Alice': 5, 'Bob': 2, 'Charlie': 3}
Winner: Alice
Rewards: {'Alice': 2, 'Bob': 2, 'Charlie': 6}
Winner: Charlie
Rewards: {'Alice': 2, 'Bob': 6, 'Charlie': 2}
Winner: Bob
Rewards: {'Alice': 5, 'Bob': 4, 'Charlie': 1}
Winner: Alice
Rewards: {'Alice': 2, 'Bob': 6, 'Charlie': 2}
Winner: Bob
Rewards: {'Alice': 7, 'Bob': 2, 'Charlie': 1}
Winner: Alice
Rewards: {'Alice': 5, 'Bob': 2, 'Charlie': 3}
Winner: Alice
Rewards: {'Alice': 2, 'Bob': 0, 'Charlie': 8}
Winner: Charlie
Rewards: {'Alice': 1, 'Bob': 3, 'Charlie

### **Monte Carlo vs. Rule-Based Player**

In [ ]:
def rule_based_move(game_state):
    legal_moves = game_state.get_legal_moves()
    # leading_suit = game_state.table_color if game_state.table_cards else None
    # suit_moves = [card for card in legal_moves if card.suit == leading_suit]

    # if suit_moves:
    #     return max(suit_moves, key=lambda card: card.rank)  # Playing the highest of the same suit

    #If no suit match, playing the highest-ranked card
    return max(legal_moves, key=lambda card: card.rank)



def evaluate_monte_carlo_rule_based(player_names,roles,index,num_games=1000, num_simulations=1000):
    monte_carlo_wins = 0
    win_results = []

    for _ in range(num_games):

        game = Game(player_names, roles,index)
        game_state = GameState(game)

        # game_state = copy.deepcopy(initial_game_state)
        game_state.current_player_index=random.choice(index)


        while not game_state.is_terminal():
            if game_state.current_player_index == 0:
                best_move = monte_carlo_best_move(game_state, num_simulations)

            else:
                legal_moves = game_state.get_legal_moves()
                best_move = rule_based_move(game_state)

            game_state.apply_move(best_move)

        rewards=game_state.tricks_won


        max_tricks = max(rewards.values())
        potential_winners = [name for name, count in rewards.items() if count == max_tricks]

        winner_name = potential_winners[0]
        print(f"Winner: {winner_name}")

        if winner_name == "Alice":
            monte_carlo_wins += 1
            win_results.append(1)
        else:
            win_results.append(0)

    win_rate = monte_carlo_wins / num_games
    std_dev = np.std(win_results, ddof=1)

    print(f"Monte Carlo vs Rule-Based Win Rate: {win_rate:.2%}")
    print(f"Standard Deviation of Wins: {std_dev:.4f}")






In [ ]:
start=time.time()
evaluate_monte_carlo_rule_based(player_names,roles,index,num_games=1000, num_simulations=50)
end=time.time()
print(f"Execution Time: {end - start:.4f} seconds")

Winner: Charlie
Winner: Bob
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Charlie
Winner: Alice
Winner: Charlie
Winner: Bob
Winner: Bob
Winner: Charlie
Winner: Bob
Winner: Charlie
Winner: Alice
Winner: Bob
Winner: Alice
Winner: Bob
Winner: Charlie
Winner: Alice
Winner: Charlie
Winner: Alice
Winner: Charlie
Winner: Alice
Winner: Bob
Winner: Alice
Winner: Charlie
Winner: Alice
Winner: Charlie
Winner: Bob
Winner: Charlie
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Bob
Winner: Bob
Winner: Alice
Winner: Charlie
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Charlie
Winner: Bob
Winner: Bob
Winner: Alice
Winner: Charlie
Winner: Bob
Winner: Alice
Winner: Charlie
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Charlie
Winner: Bob
Winner: Charlie
Winner: Charlie
Winner: Bob
Winner: Charlie
Winner: Charlie
Winner: Charlie
Winner: Bob
Winner: Charlie
Winner: Alice
Winner: Bob
Winner: Alice
Winner: Charlie
Winner: 